Code written in colabboration with Martin Hansen Skjelvareid.

In [10]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [76]:
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import matplotlib.colors
import pathlib
import tensorflow as tf
import sklearn.metrics
import pickle
import time
import skimage.filters.rank
import skimage.morphology
import sys
sys.path.insert(1, '/Users/siljegrue/Library/CloudStorage/OneDrive-UiTOffice365/Desktop/uit/Article 2023/Code/Article_2023_MASSIMAL')
import annotation, hyspec_cnn, hyspec_ml #, misc, hyspec_io, image_render, hyspec_ml, hyspec_cnn, preprocess

In [16]:
# Load RF model
rf_model = pickle.load(open("/Users/siljegrue/Library/CloudStorage/OneDrive-UiTOffice365/Desktop/uit/Article 2023/Code/Article_2023_MASSIMAL/machine_learning/RF_model.sav", 'rb'))

In [17]:
# Disable GPUs
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [26]:
# Paths
base_dir = pathlib.Path('/Volumes/MASTER_SSD/Article/Data/South')

json_gray = base_dir / 'Annotation/label_classes.json'
#hyspec_images_dir  = base_dir / '4a_Rad_Georef_SGC'
#validation_images_dir= "/Volumes/MASTER_SSD/Article/Data/South/Validation_images"
validation_images_dir = base_dir / 'Validation_Images'
#unet_model_path = base_dir / 'X_SavedKerasModels/InpaintedDataset_TestImages-12-17-24/20230511_Unet_Depth2_UsingReduceLrOnPlateau/unet_model.epoch40-loss0.028375-acc0.887.hdf5'
#rf_model_path = base_dir / 'X_SavedModels_RF_SVM/20210825_OlbergAreaS_RandomForest_InpaintedDataset.pkl'
#svm_model_path = base_dir / 'X_SavedModels_RF_SVM/20210825_OlbergAreaS_SVM_InpaintedDataset.pkl'

In [23]:
# Parameters
PAD_MULTIPLE = 2**5

# Instructions for merging / extracting classes
classes_to_merge = [['Zostera marina - dominant','Zostera marina - normal','Zostera marina - sparse'],
                    ['Rockweed','Rockweed with turf algae','Fucus serratus','Saccharina latissima'],
                    ['Turf algae - dominant','Turf algae - normal','Turf algae - sparse','Aegagropila (algae ball)']]
merged_class_names = ['Zostera marina',
                      'Rockweed',
                      'Other algae']
classes_to_extract = ['Sand',
                      'Zostera marina',
                      'Zostera marina with turf algae',
                      'Rockweed',
                      'Other algae']

# Colormap
custom_cmap = matplotlib.colors.ListedColormap(colors=['black','aqua','yellowgreen','forestgreen','brown','violet'])

In [44]:
# Read annotation metadata file, show classes
class_dict = annotation.read_hasty_metadata(json_gray)

In [78]:
# Load images, pad, merge classes 
pca_images = []
label_images = []
rgb_images = []
dataset_paths = validation_images_dir.glob('*')

for dataset_path in dataset_paths:
    # Load tensorflow datasets
    dataset = tf.data.experimental.load(str(dataset_path))
    pca_im, label_im = [d for d in dataset.as_numpy_iterator()][0]
    pca_im = hyspec_cnn.pad_image_to_multiple(pca_im,PAD_MULTIPLE)
    label_im = hyspec_cnn.pad_image_to_multiple(label_im,PAD_MULTIPLE)

FailedPreconditionError: /Volumes/MASTER_SSD/Article/Data/South/Validation_Images/OlbergholmenS_Pika_L_24-Radiance From Raw Data-Georectify Airborne Datacube/9033557713112706552/._00000000.shard/00000000.snapshot; Not a directory [Op:IteratorGetNext]